# Junk Food Classification with KNN

This notebook implements a **K-Nearest Neighbors (KNN)** classifier for image classification using a **COCO-style dataset**.  
The goal is to compare the performance of KNN against YOLO and CLIP pipelines using the same dataset and consistent evaluation metrics.

Running the pipeline is relatively quick, since it uses a pre-trained model (RestNet-50)

## Before you start

Make sure you have access to GPU. In case of any problems, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, click `Save` and try again.

In [1]:
!nvidia-smi

Sun Dec 28 21:55:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [3]:
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets


/content/datasets


## Install packages using pip

In [4]:
!pip install roboflow==1.2.11 torch==2.9.0 torchvision==0.24.0 scikit-learn==1.6.1 tqdm==4.67.1

## Download dataset from Roboflow

Don't forget to change the `API_KEY` with your dataset key.

We replicate your original dataset setup. Even though the dataset is labeled for object detection, we’ll use the full image classification approach with KNN. Labels will be derived from the most frequent class per image.

In [5]:
from roboflow import Roboflow
from google.colab import userdata

rf = Roboflow(api_key=userdata.get('ROBOFLOW_API_KEY'))
project = rf.workspace(userdata.get('ROBOFLOW_WORKSPACE_ID')).project(userdata.get('ROBOFLOW_PROJECT_ID'))
version = project.version(userdata.get('ROBOFLOW_DATASET_VERSION'))
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...


In [6]:
%cd {HOME}

/content


## Dataset Loading and Label Extraction

We will use a classification model. So, for labeling, we use two classes: junk-food-ad and non-junk-food-ad. Given the fact that the dataset is multiclass, the rule is: if there is at least one bounding box belonging to a particular image, it's junk-food-ad. Otherwise, it's non-junk-food-ad

In [7]:
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple


def load_coco_annotations(json_path: str) -> Dict:
    with open(json_path, 'r') as f:
        return json.load(f)


def process_dataset_part(
    part_dir: str,
    positive_class: str,
    negative_class: str,
    annotations_filename: str = "_annotations.coco.json"
) -> Tuple[List[str], List[str]]:
    annotations_path = os.path.join(part_dir, annotations_filename)

    if not os.path.exists(annotations_path):
        raise FileNotFoundError(f"Annotations file not found: {annotations_path}")

    # Load annotations
    coco_data = load_coco_annotations(annotations_path)

    # Create a mapping of image_id to whether it has annotations
    images_with_boxes = set()
    for annotation in coco_data['annotations']:
        images_with_boxes.add(annotation['image_id'])

    # Process images in order
    image_paths = []
    labels = []

    for image_info in coco_data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']

        image_path = os.path.join(part_dir, file_name)
        image_paths.append(image_path)

        # Assign binary label based on presence of bounding boxes
        if image_id in images_with_boxes:
            labels.append(positive_class)
        else:
            labels.append(negative_class)

    return image_paths, labels


def process_full_dataset(
    dataset_root: str,
    positive_class: str = 'junk-food-ad',
    negative_class: str = 'non-junk-food-ad',
    parts: List[str] = ['train', 'valid', 'test']
) -> Tuple[Dict[str, List[str]], Dict[str, List[str]], List[str]]:
    # Define binary classes
    classes = [positive_class, negative_class]

    all_image_paths = {}
    all_labels = {}

    for part in parts:
        part_dir = os.path.join(dataset_root, part)

        if not os.path.exists(part_dir):
            print(f"Warning: Directory not found: {part_dir}. Skipping...")
            continue

        image_paths, labels = process_dataset_part(part_dir, positive_class, negative_class)

        all_image_paths[part] = image_paths
        all_labels[part] = labels

    return all_image_paths, all_labels, classes


image_paths_dict, labels_dict, classes = process_full_dataset(dataset.location)

print("\n" + "="*50)
print("DATASET SUMMARY")
print("="*50)
print(f"\nClasses: {classes}")
print(f"\nDataset parts processed:")

for part in image_paths_dict.keys():
    print(f"\n{part.upper()}:")
    print(f"  Total images: {len(image_paths_dict[part])}")
    print(f"  Total labels: {len(labels_dict[part])}")
    print(f"  Label distribution:")
    for cls in classes:
        count = labels_dict[part].count(cls)
        percentage = (count / len(labels_dict[part]) * 100) if labels_dict[part] else 0
        print(f"    - {cls}: {count} ({percentage:.1f}%)")


# Access individual parts like this:
# train_images = image_paths_dict['train']
# train_labels = labels_dict['train']
# valid_images = image_paths_dict['valid']
# valid_labels = labels_dict['valid']
# test_images = image_paths_dict['test']
# test_labels = labels_dict['test']


DATASET SUMMARY

Classes: ['junk-food-ad', 'non-junk-food-ad']

Dataset parts processed:

TRAIN:
  Total images: 4614
  Total labels: 4614
  Label distribution:
    - junk-food-ad: 2925 (63.4%)
    - non-junk-food-ad: 1689 (36.6%)

VALID:
  Total images: 440
  Total labels: 440
  Label distribution:
    - junk-food-ad: 277 (63.0%)
    - non-junk-food-ad: 163 (37.0%)

TEST:
  Total images: 218
  Total labels: 218
  Label distribution:
    - junk-food-ad: 131 (60.1%)
    - non-junk-food-ad: 87 (39.9%)


## Feature Extraction of train set using pretrained models

KNN itself cannot extract visual features, it only compares numeric vectors.  
Therefore, we use **pre-trained** models (without their classification heads) to extract image embeddings of train set.

These embeddings (feature vectors) represent each image in a high-dimensional space that captures visual similarity.  
The extracted features are stored as a NumPy matrix and later fed into the KNN classifier.

In [8]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
import numpy as np

def extract_features(image_paths, model, feature_dim, transform, model_name):
    features = []
    with torch.no_grad():
        for path in tqdm(image_paths, desc=f"Extracting features - {model_name}"):
            try:
                img = Image.open(path).convert("RGB")
                tensor = transform(img).unsqueeze(0).to(device)
                feat = model(tensor).squeeze().cpu().numpy()
                features.append(feat)
            except Exception as e:
                print(f"Error with {path}: {e}")
                features.append(np.zeros(feature_dim))
    return np.array(features)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_configs = [
    {
        'name': 'ResNeXt-101',
        'loader': lambda: models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.DEFAULT),
        'modifier': lambda m: torch.nn.Sequential(*list(m.children())[:-1]),
        'transform': models.ResNet50_Weights.DEFAULT.transforms(),
        'feature_dim': 2048
    },
    {
        'name': 'EfficientNet V2',
        'loader': lambda: models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT),
        'modifier': lambda m: torch.nn.Sequential(*list(m.children())[:-1]),
        'transform': models.EfficientNet_V2_M_Weights.DEFAULT.transforms(),
        'feature_dim': 1280
    },
    {
        'name': 'ConvNeXt',
        'loader': lambda: models.convnext_base(weights=models.ConvNeXt_Base_Weights.DEFAULT),
        'modifier': lambda m: torch.nn.Sequential(*list(m.children())[:-1]),
        'transform': models.ConvNeXt_Base_Weights.DEFAULT.transforms(),
        'feature_dim': 1024
    },
    {
        'name': 'ViT',
        'loader': lambda: models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT),
        'modifier': lambda m: (setattr(m.heads, 'head', torch.nn.Identity()), m)[1],
        'transform': models.ViT_B_16_Weights.DEFAULT.transforms(),
        'feature_dim': 768
    },
    {
        'name': 'Swin Transformer',
        'loader': lambda: models.swin_v2_b(weights=models.Swin_V2_B_Weights.DEFAULT),
        'modifier': lambda m: torch.nn.Sequential(*list(m.children())[:-1]),
        'transform': models.Swin_B_Weights.DEFAULT.transforms(),
        'feature_dim': 1024
    },
    {
        'name': 'DINOv2',
        'loader': lambda: torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14'),
        'modifier': lambda m: m,  # No modification needed
        'transform': transforms.Compose([
            transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        'feature_dim': 768
    }
]

loaded_models = []
for config in model_configs:
    model = config['loader']()
    model = config['modifier'](model)
    model = model.to(device)
    model.eval()
    loaded_models.append({
        'model': model,
        'name': config['name'],
        'transform': config['transform'],
        'feature_dim': config['feature_dim']
    })

# Extract features for all models on train set
all_features = {}
for model_info in loaded_models:
    features = extract_features(
        image_paths_dict['train'],
        model_info['model'],
        model_info['feature_dim'],
        model_info['transform'],
        model_info['name']
    )
    all_features[model_info['name']] = features
    print(f"Feature matrix shape - {model_info['name']}: {features.shape}")

Using device: cuda


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
xFormers is not available (SwiGLU)
xFormers is not available (Attention)
xFormers is not available (Block)
Extracting features - ResNeXt-101: 100%|██████████| 4614/4614 [02:16<00:00, 33.70it/s]


Feature matrix shape - ResNeXt-101: (4614, 2048)


Extracting features - EfficientNet V2: 100%|██████████| 4614/4614 [03:09<00:00, 24.40it/s]


Feature matrix shape - EfficientNet V2: (4614, 1280)


...

Feature matrix shape - ViT: (4614, 768)


Extracting features - Swin Transformer: 100%|██████████| 4614/4614 [03:29<00:00, 21.98it/s]


Feature matrix shape - Swin Transformer: (4614, 1024)


Extracting features - DINOv2: 100%|██████████| 4614/4614 [02:05<00:00, 36.64it/s]

Feature matrix shape - DINOv2: (4614, 768)


## Training the KNN Classifiers

KNN is trained using a simple distance-based rule:
- Each image is classified based on the majority vote of its *k* nearest neighbors in the feature space.
- We use `k=5` neighbors for this experiment.

After training, we compute accuracy.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (confusion_matrix, accuracy_score, f1_score, precision_score, recall_score)

RESULTS_PATH = os.path.join(HOME, "runs", "classify")
os.makedirs(RESULTS_PATH, exist_ok=True)

encoder = LabelEncoder()
y_train = encoder.fit_transform(labels_dict['train'])

# Train KNN classifiers for all models
trained_models = {}
for model_name, features in all_features.items():
    knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
    knn.fit(features, y_train)

    trained_models[model_name] = knn
    print(f"Trained KNN for {model_name}")

Trained KNN for ResNeXt-101
Trained KNN for EfficientNet V2
Trained KNN for ConvNeXt
Trained KNN for ViT
Trained KNN for Swin Transformer
Trained KNN for DINOv2


## Metrics

In [10]:
def evaluate_model(X, y, split_name, knn, model_name):
    split_dir = os.path.join(RESULTS_PATH, split_name)
    os.makedirs(split_dir, exist_ok=True)
    y_pred = knn.predict(X)

    # We need pos_label=0 because the positive class is at 0.
    # Kinda weird, but that's how I implemented the dataset parsing and I realized the situation later on, sorry.
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, pos_label=0)
    recall = recall_score(y, y_pred, pos_label=0)
    f1 = f1_score(y, y_pred, pos_label=0)
    conf_matrix = confusion_matrix(y, y_pred)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': conf_matrix
    }

## Metrics on validation set

In [11]:
# Extract features for validation set
all_valid_features = {}
for model_info in loaded_models:
    features = extract_features(
        image_paths_dict['valid'],
        model_info['model'],
        model_info['feature_dim'],
        model_info['transform'],
        model_info['name']
    )
    all_valid_features[model_info['name']] = features

y_valid = encoder.transform(labels_dict['valid'])

# Evaluate all models on valid set
for model_name, knn in trained_models.items():
  results = evaluate_model(all_valid_features[model_name], y_valid, "valid", knn, model_name)
  print(f"{model_name} - Valid Set:")
  print(f"  Accuracy:  {results['accuracy']:.4f}")
  print(f"  Precision: {results['precision']:.4f}")
  print(f"  Recall:    {results['recall']:.4f}")
  print(f"  F1 Score:  {results['f1_score']:.4f}")
  print(f"  Confusion Matrix:")
  print(f"  {results['confusion_matrix']}")

Extracting features - DINOv2: 100%|██████████| 440/440 [00:11<00:00, 37.21it/s]


ResNeXt-101 - Valid Set:
  Accuracy:  0.8977
  Precision: 0.8893
  Recall:    0.9567
  F1 Score:  0.9217
  Confusion Matrix:
  [[265  12]
 [ 33 130]]
EfficientNet V2 - Valid Set:
  Accuracy:  0.8864
  Precision: 0.8796
  Recall:    0.9495
  F1 Score:  0.9132
  Confusion Matrix:
  [[263  14]
 [ 36 127]]
ConvNeXt - Valid Set:
  Accuracy:  0.8864
  Precision: 0.9039
  Recall:    0.9170
  F1 Score:  0.9104
  Confusion Matrix:
  [[254  23]
 [ 27 136]]
ViT - Valid Set:
  Accuracy:  0.8955
  Precision: 0.8942
  Recall:    0.9458
  F1 Score:  0.9193
  Confusion Matrix:
  [[262  15]
 [ 31 132]]
Swin Transformer - Valid Set:
  Accuracy:  0.8932
  Precision: 0.9049
  Recall:    0.9278
  F1 Score:  0.9162
  Confusion Matrix:
  [[257  20]
 [ 27 136]]
DINOv2 - Valid Set:
  Accuracy:  0.9250
  Precision: 0.9296
  Recall:    0.9531
  F1 Score:  0.9412
  Confusion Matrix:
  [[264  13]
 [ 20 143]]


## Metrics on test set

Don't forget we are trying some models, so that's why we are listing multiple metrics.

In [12]:
all_test_features = {}
for model_info in loaded_models:
    features = extract_features(
        image_paths_dict['test'],
        model_info['model'],
        model_info['feature_dim'],
        model_info['transform'],
        model_info['name']
    )
    all_test_features[model_info['name']] = features

y_test = encoder.transform(labels_dict['test'])

# Evaluate all models on test set
for model_name, knn in trained_models.items():
    results = evaluate_model(all_test_features[model_name], y_test, "test", knn, model_name)
    print(f"{model_name} - Test Set:")
    print(f"  Accuracy:  {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall:    {results['recall']:.4f}")
    print(f"  F1 Score:  {results['f1_score']:.4f}")
    print(f"  Confusion Matrix:")
    print(f"  {results['confusion_matrix']}")

Extracting features - DINOv2: 100%|██████████| 218/218 [00:05<00:00, 38.11it/s]


ResNeXt-101 - Test Set:
  Accuracy:  0.8761
  Precision: 0.8562
  Recall:    0.9542
  F1 Score:  0.9025
  Confusion Matrix:
  [[125   6]
 [ 21  66]]
EfficientNet V2 - Test Set:
  Accuracy:  0.9083
  Precision: 0.8993
  Recall:    0.9542
  F1 Score:  0.9259
  Confusion Matrix:
  [[125   6]
 [ 14  73]]
ConvNeXt - Test Set:
  Accuracy:  0.8761
  Precision: 0.8881
  Recall:    0.9084
  F1 Score:  0.8981
  Confusion Matrix:
  [[119  12]
 [ 15  72]]
ViT - Test Set:
  Accuracy:  0.8899
  Precision: 0.8905
  Recall:    0.9313
  F1 Score:  0.9104
  Confusion Matrix:
  [[122   9]
 [ 15  72]]
Swin Transformer - Test Set:
  Accuracy:  0.8807
  Precision: 0.9070
  Recall:    0.8931
  F1 Score:  0.9000
  Confusion Matrix:
  [[117  14]
 [ 12  75]]
DINOv2 - Test Set:
  Accuracy:  0.9128
  Precision: 0.9000
  Recall:    0.9618
  F1 Score:  0.9299
  Confusion Matrix:
  [[126   5]
 [ 14  73]]
